In [1]:
import pandas as pd
from datetime import datetime as dt
from sqlalchemy import create_engine
from secrets import secrets

## Update Skill Ontology list

In [ ]:
skills = []

with open("skills/new_skills.txt") as f:
    for s in f:
        skills.append(s.strip('\n'))

skills = list(set(skills))
skills.sort()
print(skills)

In [ ]:
skills_engine = create_engine(secrets['skills_db'])
df_s = pd.read_sql_query('select * from "Skill"', skills_engine)
df_s.head()

In [ ]:
ex_skill = df_s.loc[df_s['Skill'].isin(skills)]['Skill'].tolist()
print(ex_skill)

In [ ]:
skills_to_add = [s for s in skills if s not in ex_skill]
print(skills_to_add)

In [ ]:
max_id = df_s['Id'].max()
max_id

In [ ]:
df_s2 = pd.DataFrame()
df_s2.loc[:, 'Skill'] = skills_to_add
df_s2['Id'] = df_s2.index + max_id + 1
df_s2['DomainId'] = 1
df_s2.head()

In [ ]:
df_s2.to_sql('Skill', skills_engine, index=False, if_exists='append')
skills_engine.dispose()

## Update SkillStreet other skills

In [2]:
query = """
select *
from "Skill"
where "DateLastModified" = (select max("DateLastModified") from "Skill")
"""
skills_engine = create_engine(secrets['skills_db'])
df_s = pd.read_sql_query(query, skills_engine)
skills_engine.dispose()
df_s.head()

,Id,Skill,DomainId,DateCreated,DateLastModified
0,3284,AB Initio,1,2021-05-27 10:22:53,2021-05-27 10:22:53
1,3285,AWS Glue,1,2021-05-27 10:22:53,2021-05-27 10:22:53
2,3286,Airbyte,1,2021-05-27 10:22:53,2021-05-27 10:22:53
3,3287,Alooma,1,2021-05-27 10:22:53,2021-05-27 10:22:53
4,3288,Apache Accumulo,1,2021-05-27 10:22:53,2021-05-27 10:22:53


In [3]:
df_s2 = df_s[['Skill']].rename(columns={'Skill': 'name'})
df_s2['created_date'] = dt.now()
df_s2['status'] = True
df_s2['updated_date'] = dt.now()
df_s2['belong_to'] = 1
df_s2['created_user_id'] = 1
df_s2['updated_user_id'] = 1
df_s2.head()

,name,created_date,status,updated_date,belong_to,created_user_id,updated_user_id
0,AB Initio,2021-05-28 14:42:43.933669,True,2021-05-28 14:42:43.937224,1,1,1
1,AWS Glue,2021-05-28 14:42:43.933669,True,2021-05-28 14:42:43.937224,1,1,1
2,Airbyte,2021-05-28 14:42:43.933669,True,2021-05-28 14:42:43.937224,1,1,1
3,Alooma,2021-05-28 14:42:43.933669,True,2021-05-28 14:42:43.937224,1,1,1
4,Apache Accumulo,2021-05-28 14:42:43.933669,True,2021-05-28 14:42:43.937224,1,1,1


### DEV

In [ ]:
skillstreet_engine = create_engine(secrets['skillstreet_dev'])
df_s2.to_sql('other_skill_mst', skillstreet_engine, index=False, if_exists='append')
skillstreet_engine.dispose()

### STAGE

In [5]:
df_s2['created_user_id'] = -1
df_s2['updated_user_id'] = -1
df_s2.head()

,name,created_date,status,updated_date,belong_to,created_user_id,updated_user_id
0,AB Initio,2021-05-28 14:42:43.933669,True,2021-05-28 14:42:43.937224,1,-1,-1
1,AWS Glue,2021-05-28 14:42:43.933669,True,2021-05-28 14:42:43.937224,1,-1,-1
2,Airbyte,2021-05-28 14:42:43.933669,True,2021-05-28 14:42:43.937224,1,-1,-1
3,Alooma,2021-05-28 14:42:43.933669,True,2021-05-28 14:42:43.937224,1,-1,-1
4,Apache Accumulo,2021-05-28 14:42:43.933669,True,2021-05-28 14:42:43.937224,1,-1,-1


In [6]:
skillstreet_engine = create_engine(secrets['skillstreet_stage'])
df_s2.to_sql('other_skill_mst', skillstreet_engine, index=False, if_exists='append')
skillstreet_engine.dispose()

### PROD

In [7]:
df_s2['created_user_id'] = 1
df_s2['updated_user_id'] = 1
df_s2.head()

,name,created_date,status,updated_date,belong_to,created_user_id,updated_user_id
0,AB Initio,2021-05-28 14:42:43.933669,True,2021-05-28 14:42:43.937224,1,1,1
1,AWS Glue,2021-05-28 14:42:43.933669,True,2021-05-28 14:42:43.937224,1,1,1
2,Airbyte,2021-05-28 14:42:43.933669,True,2021-05-28 14:42:43.937224,1,1,1
3,Alooma,2021-05-28 14:42:43.933669,True,2021-05-28 14:42:43.937224,1,1,1
4,Apache Accumulo,2021-05-28 14:42:43.933669,True,2021-05-28 14:42:43.937224,1,1,1


In [8]:
skillstreet_engine = create_engine(secrets['skillstreets_prod'])
df_s2.to_sql('other_skill_mst', skillstreet_engine, index=False, if_exists='append')
skillstreet_engine.dispose()